In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model , Sequential
from keras.layers import Input, LSTM , Bidirectional , Dropout , Activation , Dense , Add , GRU
from keras.layers import Conv1D , MaxPooling1D , Flatten , AveragePooling1D , concatenate , BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from keras.utils import multi_gpu_model
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ' 0 , 1 '


Bad key "text.kerning_factor" on line 4 in
/home/Yan-Cheng-Hsu/anaconda3/envs/test/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
Using TensorFlow backend.


In [2]:
def rmse_loss (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

In [3]:
print( 'loading data....' )
ppg_part1 = np.load( '/data1/Yan-Cheng-Hsu/ppgData/dataperC/ppg_part1.npy' , allow_pickle = True )
ppg_part2 = np.load( '/data1/Yan-Cheng-Hsu/ppgData/dataperC/ppg_part2.npy' , allow_pickle = True )
ppg_part3 = np.load( '/data1/Yan-Cheng-Hsu/ppgData/dataperC/ppg_part3.npy' , allow_pickle = True )
sbp = np.load( '/data1/Yan-Cheng-Hsu/ppgData/dataperC/sbp.npy' , allow_pickle = True )
dbp = np.load( '/data1/Yan-Cheng-Hsu/ppgData/dataperC/dbp.npy' , allow_pickle = True )
print( 'loading data finished.' )

loading data....
loading data finished.


In [4]:
print('data preprocessing......')
ppg_min = 100
ppg_max = -100

for x in range( len(ppg_part1) ):
    for i in range( len(ppg_part1[x]) ):
        temp_min = np.min( ppg_part1[x][i] )
        temp_max = np.max( ppg_part1[x][i] )
        if temp_min < ppg_min:
            ppg_min = temp_min
        if temp_max > ppg_max:
            ppg_max = temp_max
for x in range( len(ppg_part2) ):
    for i in range( len(ppg_part2[x]) ):
        temp_min = np.min( ppg_part2[x][i] )
        temp_max = np.max( ppg_part2[x][i] )
        if temp_min < ppg_min:
            ppg_min = temp_min
        if temp_max > ppg_max:
            ppg_max = temp_max
for x in range( len(ppg_part3) ):
    for i in range( len(ppg_part3[x]) ):
        temp_min = np.min( ppg_part3[x][i] )
        temp_max = np.max( ppg_part3[x][i] )
        if temp_min < ppg_min:
            ppg_min = temp_min
        if temp_max > ppg_max:
            ppg_max = temp_max

            
ppg_part1 = ( ppg_part1 - ppg_min ) / ( ppg_max - ppg_min )
ppg_part2 = ( ppg_part2 - ppg_min ) / ( ppg_max - ppg_min )
ppg_part3 = ( ppg_part3 - ppg_min ) / ( ppg_max - ppg_min )


data preprocessing......


In [5]:
ppg = []

In [6]:
for x in range( len(ppg_part1) ):
    for i in range( len(ppg_part1[x]) ):
        ppg.append( ppg_part1[x][i] )
        
for x in range( len(ppg_part2) ):
    for i in range( len(ppg_part2[x]) ):
        ppg.append( ppg_part2[x][i] )
for x in range( len(ppg_part3) ):
    for i in range( len(ppg_part3[x]) ):
        ppg.append( ppg_part3[x][i] )
ppg = np.array( ppg )

In [7]:
ppg.shape

(2468014, 300)

In [8]:
bp = []

for x in range( len(sbp) ):
    for i in range( len(sbp[x]) ):
        temp = []
        temp.append( sbp[x][i] )
        temp.append( dbp[x][i] )
        temp = np.array( temp )
        bp.append( temp )
bp = np.array( bp )

endIndex = int( len(ppg)*0.8 )
ppg_train = ppg[0:endIndex]
ppg_test = ppg[endIndex:]
bp_train = bp[0:endIndex]
bp_test = bp[endIndex:]

PPG_train = ppg_train.reshape( len(ppg_train) , 300 , 1 ).astype( float )
PPG_test = ppg_test.reshape( len(ppg_test) , 300 , 1 ).astype( float )
BP_train = bp_train.reshape( len(bp_train) , 1 , 2 ).astype( float )
BP_test = bp_test.reshape( len(bp_test) , 1 , 2 ).astype( float )

print('data preprocessing finished.')


KeyboardInterrupt: 

In [ ]:
print( PPG_train.shape )
print( PPG_test.shape )
print( BP_train.shape )
print( BP_test.shape )

In [ ]:
#model1 start
Inputshape = ( len(PPG_train[0]) , len(PPG_train[0][0]) )
X_input1 = Input( Inputshape )
X_CNN1 = Conv1D( 64 , int(len( PPG_train[0] )/4) , border_mode = 'same' , activation = 'relu' )( X_input1 )
X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN1 )
X_CNN1 = Conv1D( 64 , 4 , border_mode = 'same' , activation = 'relu' )( X_CNN1 ) 
X_CNN1 = X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN1 )
X_CNN1 = Conv1D( 128 , 4 , border_mode = 'same' , activation = 'relu' )( X_CNN1 ) 
X_CNN1 = X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN1 )
X_CNN1 = Dropout(0.25)(X_CNN1)

#X_CNN1 = BatchNormalization( X_CNN1 )



''''X_input2 = Input( Inputshape )
X_CNN2 = Conv1D( 64 , int(len( PPG_train[0] )/2) , border_mode = 'same' , activation = 'relu' )( X_input2 )
X_CNN2 = MaxPooling1D( pool_size = 2 )( X_CNN2 )
X_CNN2 = Conv1D( 64 , 4 , border_mode = 'same' , activation = 'relu' )( X_CNN2 ) 
X_CNN2 = X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN2 )
X_CNN2 = Conv1D( 128 , 4 , border_mode = 'same' , activation = 'relu' )( X_CNN2 ) 
X_CNN2 = X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN2 )
X_CNN2 = Dropout(0.25)(X_CNN2)
X_CNN2 = BatchNormalization( X_CNN2 )

X_input3 = Input( Inputshape )
X_CNN3 = Conv1D( 64 , int(len( PPG_train[0] )/2) , border_mode = 'same' , activation = 'relu' )( X_input3 )
X_CNN3 = MaxPooling1D( pool_size = 2 )( X_CNN3 )
X_CNN3 = Conv1D( 64 , 4 , border_mode = 'same' , activation = 'relu' )( X_CNN3 ) 
X_CNN3 = X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN3 )
X_CNN3 = Conv1D( 128 , 4 , border_mode = 'same' , activation = 'relu' )( X_CNN3 ) 
X_CNN3 = X_CNN1 = MaxPooling1D( pool_size = 2 )( X_CNN3 )
X_CNN3 = Dropout(0.25)(X_CNN3)
X_CNN3 = BatchNormalization( X_CNN3 )


X_CNN = concatenate( [ X_CNN1 , X_CNN2 , X_CNN3 ] )'''


LayerUnits = 1024*2
''''X0 = Bidirectional(GRU(LayerUnits, return_sequences=True), merge_mode='concat')(X_CNN1)
X0 = Dropout(0.2)(X0)
X0 = Activation('relu')(X0)

X = GRU(LayerUnits, return_sequences=True)(X0)
X = Dropout(0.2)(X)
X = Activation('relu')(X)

X1 = GRU(LayerUnits, return_sequences=True)(X)
X1 = Dropout(0.2)(X1)
X1 = Activation('relu')(X1)

X2 = concatenate([X, X1])
X2 = GRU(LayerUnits, return_sequences=True)(X2)
X2 = Dropout(0.2)(X2)
X2 = Activation('relu')(X2)


X3 = concatenate( [X2,X] )
X3 = GRU(LayerUnits, return_sequences=True)(X3)
X3 = Dropout(0.2)(X3)
X3 = Activation('relu')(X3)'''

X = Dense(LayerUnits , activation = 'relu')(X_CNN1)
X = Dropout(0.2)(X)
X = Dense(LayerUnits , activation = 'relu')(X_CNN1)
X = Dropout(0.2)(X)
X = Dense(LayerUnits , activation = 'relu')(X_CNN1)
X = Dropout(0.2)(X)
X = Dense(LayerUnits , activation = 'relu')(X_CNN1)
X = Dropout(0.2)(X)
X = Dense(LayerUnits , activation = 'relu')(X_CNN1)
X = Dropout(0.2)(X)

X_output = Dense(2)(X)

GRUModel = Model( inputs = X_input1  , outputs = X_output )
# Compiling
GRUModel.compile(optimizer = 'adam', loss = rmse_loss )
GRUModel.summary()
GRUModel.fit( PPG_train , BP_train, validation_split = 0.1 , epochs = 100 , batch_size = 2048)

path = '/data1/Yan-Cheng-Hsu/CNNModel/model_20200516/CNN_model_20200516_1.h5'
print( 'saving model.....' )
GRUModel.save( path )
print( 'finish saving model.....' )



print( 'Training set : '  ,  GRUModel.evaluate( PPG_test , BP_test ) )
print( 'Testing set :' , GRUModel.evaluate( PPG_test , BP_test ) )

